In [12]:
from pathlib import Path
import re
import pandas as pd
import sys
import os.path as op

In [13]:

ANNOT_DIR = Path("dset/annotations")
OUT_DIR = Path("dset/derivatives/annotations")
OUT_DIR.mkdir(parents=True, exist_ok=True)


In [14]:
def find_annotation_files(annotation_dir: Path):
    pattern = re.compile(r"^(S\d+E\d+R\d+)_([A-Za-z]{2})\.csv$")
    groups = {}
    for p in sorted(annotation_dir.iterdir()):
        if not p.is_file():
            continue
        m = pattern.match(p.name)
        if not m:
            continue
        prefix = m.group(1)
        annot = m.group(2)
        groups.setdefault(prefix, []).append((annot, p))
    return groups

In [15]:
def detect_columns(df: pd.DataFrame):
    # detect index column
    if "index" in df.columns:
        idx_col = "index"
    else:
        # fallback: first column
        idx_col = df.columns[0]

    # detect valence and arousal columns (case-insensitive)
    val_cols = [c for c in df.columns if "valence" in c.lower()]
    aro_cols = [c for c in df.columns if "arousal" in c.lower()]

    val_col = val_cols[0] if val_cols else None
    aro_col = aro_cols[0] if aro_cols else None
    return idx_col, val_col, aro_col

In [16]:
def combine_group(prefix, files):
    # files: list of (annotator_code, Path)
    
    # Skip if less than 2 files
    if len(files) < 2:
        print(f"Skipping {prefix}: need at least 2 files, found {len(files)}")
        return None
    
    dfs = []
    renamed = []
    valid_count = 0  # Track valid files separately from loop index
    
    for annot, path in sorted(files):
        df = pd.read_csv(path)
        idx_col, val_col, aro_col = detect_columns(df)
        if val_col is None or aro_col is None:
            print(f"Skipping {path.name}: could not find valence/arousal columns", file=sys.stderr)
            continue

        # create minimal df using valid_count for consistent numbering
        valid_count += 1
        out = pd.DataFrame()
        out["index"] = df[idx_col]
        out[f"valence_{valid_count}"] = df[val_col].values
        out[f"arousal_{valid_count}"] = df[aro_col].values
        dfs.append(out)
        renamed.append((annot, f"valence_{valid_count}", f"arousal_{valid_count}"))

    if len(dfs) < 2:
        print(f"Skipping {prefix}: need at least 2 valid files after filtering, found {len(dfs)}")
        return None

    # merge on index
    merged = dfs[0]
    for d in dfs[1:]:
        merged = pd.merge(merged, d, on="index", how="outer")

    # sort by index if numeric
    try:
        merged["index"] = pd.to_numeric(merged["index"])
        merged = merged.sort_values("index").reset_index(drop=True)
    except Exception:
        merged = merged.reset_index(drop=True)

    # Reorder columns: index, all valence_*, then all arousal_*
    val_cols = [c for c in merged.columns if c.startswith("valence_")]
    aro_cols = [c for c in merged.columns if c.startswith("arousal_")]

    def _num_suffix(colname):
        parts = colname.split("_")
        try:
            return int(parts[-1])
        except Exception:
            return 0

    val_cols = sorted(val_cols, key=_num_suffix)
    aro_cols = sorted(aro_cols, key=_num_suffix)

    new_cols = ["index"] + val_cols + aro_cols
    # keep any unexpected columns at the end (shouldn't typically occur)
    tail = [c for c in merged.columns if c not in new_cols]
    merged = merged[new_cols + tail]

    # Remove rows with insufficient ratings and document removals
    initial_rows = len(merged)
    removed_clips = []
    
    # Check each row for sufficient ratings and valid scores
    rows_to_keep = []
    for i, row in merged.iterrows():
        # Count non-null valence ratings and check for scores > 7
        val_ratings = []
        val_invalid_scores = []
        for col in val_cols:
            if pd.notna(row[col]):
                try:
                    score = float(row[col])  # Convert to numeric
                    if score > 7:
                        val_invalid_scores.append(score)
                    else:
                        val_ratings.append(score)
                except (ValueError, TypeError):
                    # Skip non-numeric values
                    print(f"Warning: Non-numeric valence value {row[col]} in {col} for index {row['index']}")
                    continue
        
        # Count non-null arousal ratings and check for scores > 7
        aro_ratings = []
        aro_invalid_scores = []
        for col in aro_cols:
            if pd.notna(row[col]):
                try:
                    score = float(row[col])  # Convert to numeric
                    if score > 7:
                        aro_invalid_scores.append(score)
                    else:
                        aro_ratings.append(score)
                except (ValueError, TypeError):
                    # Skip non-numeric values
                    print(f"Warning: Non-numeric arousal value {row[col]} in {col} for index {row['index']}")
                    continue
        
        # Determine removal reason
        removal_reasons = []
        if len(val_ratings) < 2:
            removal_reasons.append(f"insufficient valence ratings ({len(val_ratings)})")
        if len(aro_ratings) < 2:
            removal_reasons.append(f"insufficient arousal ratings ({len(aro_ratings)})")
        if val_invalid_scores:
            removal_reasons.append(f"valence scores > 7: {val_invalid_scores}")
        if aro_invalid_scores:
            removal_reasons.append(f"arousal scores > 7: {aro_invalid_scores}")
        
        # Keep row only if it has at least 2 valid ratings for both valence and arousal
        if len(val_ratings) >= 2 and len(aro_ratings) >= 2 and not val_invalid_scores and not aro_invalid_scores:
            rows_to_keep.append(i)
        else:
            removed_clips.append({
                'index': row['index'],
                'valence_ratings': len(val_ratings),
                'arousal_ratings': len(aro_ratings),
                'valence_invalid_scores': len(val_invalid_scores),
                'arousal_invalid_scores': len(aro_invalid_scores),
                'reason': "; ".join(removal_reasons)
            })
    
    # Filter the dataframe
    if rows_to_keep:
        merged_filtered = merged.iloc[rows_to_keep].reset_index(drop=True)
    else:
        merged_filtered = pd.DataFrame(columns=merged.columns)
    
    final_rows = len(merged_filtered)
    
    # Report removals
    if removed_clips:
        print(f"  Removed {len(removed_clips)}/{initial_rows} clips with insufficient ratings:")
        for clip in removed_clips[:5]:  # Show first 5 removals
            print(f"    - {clip['index']}: {clip['reason']}")
        if len(removed_clips) > 5:
            print(f"    ... and {len(removed_clips) - 5} more")
    
    return merged_filtered, renamed, removed_clips

In [17]:
groups = find_annotation_files(ANNOT_DIR)

summary = []
all_removals = []

for prefix, files in groups.items():
    out = combine_group(prefix, files)
    if out is None:
        print(f"No valid files for {prefix}")
        continue
    
    merged, renamed, removed_clips = out
    
    # Save the combined file
    out_path = OUT_DIR / f"{prefix}.csv"
    merged.to_csv(out_path, index=False)
    print(f"Wrote {out_path} ({len(files)} files -> {len(merged)} rows)")
    
    # Track removals for documentation
    if removed_clips:
        for clip in removed_clips:
            clip['run'] = prefix
        all_removals.extend(removed_clips)
    
    summary.append((prefix, len(files), out_path, len(removed_clips)))

print("\nFinished. Combined groups:")
for s in summary:
    print(f"  {s[0]}: {s[1]} files -> {s[2]} ({s[3]} clips removed)")

# Save removal documentation
if all_removals:
    removal_df = pd.DataFrame(all_removals)
    removal_path = OUT_DIR / "removed_clips_log.csv"
    removal_df.to_csv(removal_path, index=False)
    print(f"\nRemoval log saved to: {removal_path}")
    print(f"Total clips removed across all runs: {len(all_removals)}")
    
    # Summary by reason
    removal_summary = removal_df.groupby('reason').size().reset_index(name='count')
    print(f"\nRemoval reasons:")
    for _, row in removal_summary.iterrows():
        print(f"  {row['reason']}: {row['count']} clips")
else:
    print("\nNo clips were removed - all had sufficient ratings.")

  Removed 36/334 clips with insufficient ratings:
    - S01E01R01_clip0029: insufficient arousal ratings (1); arousal scores > 7: [10.0]
    - S01E01R01_clip0030: insufficient arousal ratings (1); arousal scores > 7: [10.0]
    - S01E01R01_clip0031: insufficient arousal ratings (1); arousal scores > 7: [9.0]
    - S01E01R01_clip0032: insufficient arousal ratings (1); arousal scores > 7: [9.0]
    - S01E01R01_clip0033: insufficient arousal ratings (1); arousal scores > 7: [9.0]
    ... and 31 more
Wrote dset/derivatives/annotations/S01E01R01.csv (2 files -> 298 rows)
Wrote dset/derivatives/annotations/S01E01R02.csv (2 files -> 276 rows)
  Removed 48/275 clips with insufficient ratings:
    - S01E01R03_clip0147: insufficient arousal ratings (1); arousal scores > 7: [8.0]
    - S01E01R03_clip0148: insufficient arousal ratings (1); arousal scores > 7: [8.0]
    - S01E01R03_clip0149: insufficient arousal ratings (1); arousal scores > 7: [8.0]
    - S01E01R03_clip0150: insufficient arousal r

In [18]:
# Combine runs by episode to create episode-level CSVs
episode_groups = {}

# Group the existing combined CSVs by episode
for prefix, file_count, file_path, removed_count in summary:
    # Extract episode from prefix (e.g., S01E01R01 -> S01E01)
    episode_match = re.match(r"(S\d+E\d+)R\d+", prefix)
    if episode_match:
        episode = episode_match.group(1)
        episode_groups.setdefault(episode, []).append((prefix, file_path))

print(f"Found {len(episode_groups)} episodes:")
for episode, runs in episode_groups.items():
    print(f"  {episode}: {len(runs)} runs")

# Create combined episode files
episode_summary = []
for episode, runs in episode_groups.items():
    print(f"\nCombining runs for {episode}:")
    
    episode_dfs = []
    for run_prefix, run_path in sorted(runs):
        # Load the run data
        run_df = pd.read_csv(run_path)
        
        # Add run identifier column
        run_df['run'] = run_prefix
        episode_dfs.append(run_df)
        print(f"  Added {run_prefix}: {len(run_df)} clips")
    
    if episode_dfs:
        # Concatenate all runs for this episode
        episode_combined = pd.concat(episode_dfs, ignore_index=True)
        
        # Reorder columns to put run first, then index, then emotions
        val_cols = [c for c in episode_combined.columns if c.startswith("valence_")]
        aro_cols = [c for c in episode_combined.columns if c.startswith("arousal_")]
        other_cols = [c for c in episode_combined.columns if c not in ['run', 'index'] + val_cols + aro_cols]
        
        new_order = ['run', 'index'] + val_cols + aro_cols + other_cols
        episode_combined = episode_combined[new_order]
        
        # Save episode-level file
        episode_path = OUT_DIR / f"{episode}.csv"
        episode_combined.to_csv(episode_path, index=False)
        
        total_clips = len(episode_combined)
        episode_summary.append((episode, len(runs), episode_path, total_clips))
        print(f"  Saved {episode_path}: {total_clips} total clips across {len(runs)} runs")

print(f"\nEpisode-level files created:")
for episode, run_count, file_path, clip_count in episode_summary:
    print(f"  {episode}: {run_count} runs, {clip_count} clips -> {file_path}")

Found 4 episodes:
  S01E01: 3 runs
  S01E02: 7 runs
  S01E03: 6 runs
  S01E04: 5 runs

Combining runs for S01E01:
  Added S01E01R01: 298 clips
  Added S01E01R02: 276 clips
  Added S01E01R03: 227 clips
  Saved dset/derivatives/annotations/S01E01.csv: 801 total clips across 3 runs

Combining runs for S01E02:
  Added S01E02R01: 303 clips
  Added S01E02R02: 230 clips
  Added S01E02R03: 356 clips
  Added S01E02R04: 350 clips
  Added S01E02R05: 292 clips
  Added S01E02R06: 192 clips
  Added S01E02R07: 281 clips
  Saved dset/derivatives/annotations/S01E02.csv: 2004 total clips across 7 runs

Combining runs for S01E03:
  Added S01E03R01: 261 clips
  Added S01E03R02: 309 clips
  Added S01E03R03: 316 clips
  Added S01E03R04: 359 clips
  Added S01E03R05: 268 clips
  Added S01E03R06: 408 clips
  Saved dset/derivatives/annotations/S01E03.csv: 1921 total clips across 6 runs

Combining runs for S01E04:
  Added S01E04R01: 307 clips
  Added S01E04R02: 296 clips
  Added S01E04R03: 293 clips
  Added S01E

In [19]:
episode3_df = pd.read_csv(op.join(OUT_DIR, "S01E03.csv"))

In [20]:
# Comprehensive analysis of episode 3 data
print("="*60)
print("EPISODE 3 DATA ANALYSIS")
print("="*60)

print(f"DataFrame shape: {episode3_df.shape}")
print(f"Columns: {list(episode3_df.columns)}")
print(f"Data types:\n{episode3_df.dtypes}")

print("\n" + "="*40)
print("SAMPLE DATA (first 10 rows):")
print("="*40)
print(episode3_df.head(10))

print("\n" + "="*40)
print("INDEX COLUMN ANALYSIS:")
print("="*40)
print(f"Index column type: {type(episode3_df['index'].iloc[0])}")
print(f"Index unique values (first 20): {episode3_df['index'].unique()[:20]}")
print(f"Any null values in index: {episode3_df['index'].isnull().sum()}")

# Try the index processing that caused issues
print("\n" + "="*40)
print("INDEX PROCESSING TEST:")
print("="*40)
try:
    index_as_str = episode3_df['index'].astype(str)
    print(f"Index as string (first 10): {index_as_str.head(10).tolist()}")
    
    last_4_chars = index_as_str.str[-4:]
    print(f"Last 4 characters (first 10): {last_4_chars.head(10).tolist()}")
    
    last_4_as_int = last_4_chars.astype(int)
    print(f"Last 4 as int (first 10): {last_4_as_int.head(10).tolist()}")
    print("✅ Index processing successful")
except Exception as e:
    print(f"❌ Index processing failed: {e}")
    print(f"Error type: {type(e).__name__}")

print("\n" + "="*40)
print("EMOTION COLUMNS ANALYSIS:")
print("="*40)
val_cols = [col for col in episode3_df.columns if 'valence' in col.lower()]
aro_cols = [col for col in episode3_df.columns if 'arousal' in col.lower()]

print(f"Valence columns: {val_cols}")
print(f"Arousal columns: {aro_cols}")

for col in val_cols + aro_cols:
    print(f"\n{col}:")
    print(f"  Type: {episode3_df[col].dtype}")
    print(f"  Unique values: {sorted(episode3_df[col].unique())}")
    print(f"  Null count: {episode3_df[col].isnull().sum()}")
    print(f"  Sample values: {episode3_df[col].head(10).tolist()}")

print("\n" + "="*40)
print("RUN COLUMN ANALYSIS:")
print("="*40)
if 'run' in episode3_df.columns:
    print(f"Run column unique values: {episode3_df['run'].unique()}")
    print(f"Run value counts:\n{episode3_df['run'].value_counts()}")
else:
    print("No 'run' column found")

print("\n" + "="*40)
print("DATA QUALITY CHECKS:")
print("="*40)
print(f"Any completely empty rows: {episode3_df.isnull().all(axis=1).sum()}")
print(f"Any duplicate rows: {episode3_df.duplicated().sum()}")

# Check for problematic characters or mixed types
print(f"\nChecking for mixed data types in emotion columns:")
for col in val_cols + aro_cols:
    try:
        numeric_version = pd.to_numeric(episode3_df[col], errors='coerce')
        nan_count = numeric_version.isnull().sum()
        original_nan_count = episode3_df[col].isnull().sum()
        conversion_issues = nan_count - original_nan_count
        if conversion_issues > 0:
            print(f"  {col}: {conversion_issues} values couldn't convert to numeric")
            # Show the problematic values
            mask = pd.to_numeric(episode3_df[col], errors='coerce').isnull() & episode3_df[col].notnull()
            problematic_values = episode3_df.loc[mask, col].unique()
            print(f"    Problematic values: {problematic_values}")
        else:
            print(f"  {col}: ✅ All values convert to numeric properly")
    except Exception as e:
        print(f"  {col}: ❌ Error checking numeric conversion: {e}")

print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print("="*60)

EPISODE 3 DATA ANALYSIS
DataFrame shape: (1921, 6)
Columns: ['run', 'index', 'valence_1', 'valence_2', 'arousal_1', 'arousal_2']
Data types:
run           object
index         object
valence_1    float64
valence_2      int64
arousal_1    float64
arousal_2      int64
dtype: object

SAMPLE DATA (first 10 rows):
         run               index  valence_1  valence_2  arousal_1  arousal_2
0  S01E03R01  S01E03R01_clip0000        3.0          5        3.0          4
1  S01E03R01  S01E03R01_clip0001        3.0          5        3.0          4
2  S01E03R01  S01E03R01_clip0002        3.0          5        3.0          4
3  S01E03R01  S01E03R01_clip0003        1.0          5        7.0          5
4  S01E03R01  S01E03R01_clip0004        1.0          5        7.0          5
5  S01E03R01  S01E03R01_clip0005        1.0          5        7.0          5
6  S01E03R01  S01E03R01_clip0006        1.0          5        5.0          5
7  S01E03R01  S01E03R01_clip0007        1.0          5        5.0        